In [1]:
'''
ssg convert process용 임시
-ep download
-convert to feed
-feed upload

*대용량 카탈로그의 경우
-피드 제품수 제한 (900만건 이상에서 오류남..) 
-facebook api 업로드 방식은 truncate insert / update only 두가지
-피드 나누어 제품 업로드해야함
-증분 업데이트시 개별 피드에 모두 해줘야함

*일단구현
-feed write시 
-카탈로그에 나눠진 피드 upload
-증분 업데이트 추가해야함
-증분 업데이트는 피드별로 모두 upload(update_only)

*ssg의 경우
-카탈로그1 -> 카탈로그2 수정만해서 재배포 (링크값)
-1,2 카탈로그 증분 업데이트
-머천센터 업로드는 500만건으로 고정되어있음(확인?)
-custom 링크/제외카테고리/이미지사이즈
'''

'\nssg convert process용 임시\n-ep download\n-convert to feed\n-feed upload\n\n*대용량 카탈로그의 경우\n-피드 제품수 제한 (900만건 이상에서 오류남..) \n-facebook api 업로드 방식은 truncate insert / update only 두가지\n-피드 나누어 제품 업로드해야함\n-증분 업데이트시 개별 피드에 모두 해줘야함\n\n*일단구현\n-feed write시 \n-카탈로그에 나눠진 피드 upload\n-증분 업데이트 추가해야함\n-증분 업데이트는 피드별로 모두 upload(update_only)\n\n*ssg의 경우\n-카탈로그1 -> 카탈로그2 수정만해서 재배포 (링크값)\n-1,2 카탈로그 증분 업데이트\n-머천센터 업로드는 500만건으로 고정되어있음(확인?)\n-custom 링크/제외카테고리/이미지사이즈\n'

## Import

In [11]:
import pandas as pd
import os
import gc
import zipfile
import requests
import asyncio, aiohttp, aiofiles
from starlette.config import Config
from urllib import parse
import json, time, datetime

## config

In [2]:
# ### ssg_eppe
config = {    
    "info" : {
        "media" : "facebook",
        "catalog_id" : "268046537186348",        
        "feed_id" : ["2499714026735797","457448645680512","887702468681795","933601424073448","433956697833525"],
        "name" : "ssg_ep_segment_test"
    },
    "ep" : {
        "url" : "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/test/data/ssg_facebookNoCkwhereEpAll.csv",
        "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/ep/ep_268046537186348.csv",
        "format" : "csv",
        "sep" : ",",
        "encoding" : "utf-16",
        "cron" : ""
    },
    "ep_update" : {
        "url" : "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/test/data/ssg_facebookNoCkwhereEpBrief.csv",
        "path" : "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/ep/ep_268046537186348_update.csv",
        "format" : "csv",
        "sep" : ",",
        "encoding" : "utf-16",
        "cron" : ""
    },
    "feed" : {
        "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/feed_268046537186348.tsv",
    },
    "feed_update" : {
        "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/feed_268046537186348_update.tsv"
    },
    "columns" : {
        "id" : "id",
        "title" : "title",
        "link" : "link",
        "image_link" : "image_link",
        "price" : "value",
        "brand" : "brand",
        "description" : "description",
        "availability" : "availability",
        "condition" : "condition"
    },
    "status" : ""
}

In [103]:
### ssg_eppe_ios
# update는 config새로 만들고 status값 추가
# config = {    
#     "info" : {
#         "media" : "facebook",
#         "catalog_id" : "225456985373646",        
#         "feed_id" : ["2433331320101804","704765896857210","136652408333414","250350879959740","2477650072531024"],
#         "name" : "ssg_ep_ios_segment_test"
#     },
#     "ep" : {
#         "url" : "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/test/data/ssg_facebookNoCkwhereEpAll.csv",
#         "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/ep/ep_268046537186348.csv",        
#         "format" : "csv",
#         "sep" : ",",
#         "encoding" : "utf-16"
#     },    
#     "feed" : {
#         "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/feed_225456985373646.tsv",        
#     },
#     "columns" : {
#         "id" : "id",
#         "title" : "title",
#         "link" : "link",
#         "image_link" : "image_link",
#         "price" : "value",
#         "brand" : "brand",
#         "description" : "description",
#         "availability" : "availability",
#         "condition" : "condition"
#     }    
# }

In [104]:
# config = {    
#     "info" : {
#         "media" : "facebook",        
#         "catalog_id" : "432089861336082", # test_선상협
#         "feed_id" : ["236164118048821","254210579552181"],
#         "name" : "test_feed_upload"
#     },
#     "ep" : {        
#         "url" : "http://ep.hellonature.co.kr/PGD/EP/NAVER/all.txt",
#         "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/ep/ep_432089861336082.csv",
#         "format" : "tsv",
#         "sep" : "\t",
#         "encoding" : "utf-8"
#     },
#     "feed" : {        
#         "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/feed_432089861336082.tsv",
#     },
#     "columns" : {
#          "id" : "id",
#         "title" : "title",
#         "link" : "link",
#         "image_link" : "image_link",
#         "price_pc" : "value",
#         "brand" : "brand"
#     }    
# }

## Util

In [3]:
'''
다운로드시 파일 작성시간 확인 체크 필요
'''
async def epDownload(fromUrl, toPath):
    async with aiohttp.ClientSession() as session:
        async with session.get(fromUrl) as response:
            chunk_size = 1024*1024*10 # 10MB
            async with aiofiles.open(toPath, 'wb') as f:
                while True:
                    chunk = await response.content.read(chunk_size)                        
                    if not chunk : break
                    await f.write(chunk)

                # result = self.getInfo(toPath)
                # self.logger.info('Download complete '+str(result))
                # return result

In [4]:
def epLoad():
    # 원본 컬럼리스트 확인
    columns = pd.read_csv(
        config['ep']['path'], 
        nrows=1, sep=config['ep']['sep'], 
        encoding=config['ep']['encoding']
        )
    columns = list(columns)

    result = pd.read_csv(
        config['ep']['path'],
        nrows=None,
        chunksize=100000,
        header=0,
        dtype=str,
        sep=config['ep']['sep'],
        error_bad_lines=False,
        usecols=columns,
        encoding=config['ep']['encoding']
        )
    return result


In [5]:
async def feedWrite(num, feedPath, df):
    # print(feedPath)
    if num == 0:
        mode='w' # 새로쓰기
        header=True 
    else:
        mode='a' # 이어쓰기
        header=False
        
    df.to_csv(feedPath, 
                index=False, # 자체 인덱스제거
                sep='\t', 
                mode=mode,
                header=header, # 컬럼명 
                encoding='utf-8')

In [6]:
'''
feed파일 로컬에선 테스트불가
'''
async def facebookUpload( feed_id, feed_url, isUpdateOnly='false'):
    access_token = 'EAACpvLy2z30BAKxprnqZBpE8pfa08FVn6ya32d9TQk6wbXC4fmINmN8MVBur2eKGtYysSmMXMkttVg6s1bWu2yscUwEEZAdjJpRXyjxThbcuRYcKeSPMQm8ZAWhp4l5d6urUfxcjcY0UEVf46TpfhdlmSdKvlKbmxTIm6sc3xn2kIIWU6VX'
    api_url = f'https://graph.facebook.com/v9.0/{feed_id}/uploads'
    params ={'update_only': isUpdateOnly ,'access_token': access_token, 'url': feed_url}
    async with aiohttp.ClientSession() as session:
        async with session.post(api_url, data=params) as response:
            result = await response.text()            
            return result 

In [7]:
def convertFilter(dataFrame):
    result = dataFrame

    keys = list(config['columns'].keys())
    result = result[keys]
    result.rename(columns=config['columns'], inplace=False)
    
    # 공백제거
    for key in keys:
        result[key] = result[key].str.strip()

    # 기본값
    if 'availability' not in result.columns :
        result['availability'] = 'in stock'
    if 'condition' not in result.columns : 
        result['condition'] = 'new'
    if 'description' not in result.columns :
        result['description'] = result['title'].str.lower()
    else : 
        result['description'] = result['description'].str.lower()

    ########################
    ## custom filter 작성 ##
    ########################
    catalog_id = config['info']['catalog_id']


    # SSG_EPPE / SSG_EPPE_IOS
    if catalog_id == '268046537186348' or catalog_id == '225456985373646':                
        # 제외 title
        stopword = ['마스크','소독제','손소독겔','새니타이저','손살균제','snitizers','disinfection wipes']
        result = result[~result['title'].str.contains('|'.join(stopword))]

        # 이미지사이즈 조정?
        result['image_link'] = result['image_link'].str.replace('_500','_600', regex=False)

        # link값 변경        
        if catalog_id=='268046537186348' : # ssg_eppe            
            result['link'] = 'https://ad.adpool.co.kr/app/ssg/item/'+result['id'] # f''으로 적용하면 axis가 안맞는듯?

        elif catalog_id=='225456985373646' : # ssg_eppe_ios            
            result['link'] = 'http://m.ssg.com/item/itemView.ssg?itemId='+result['id']+'&gateYn=Y&mobilAppSvcNo=3'
                        

    elif catalog_id == '':
        pass

    return result

In [8]:
def zipped(fromPath, toPath):
    os.makedirs(os.path.dirname(toPath), exist_ok=True) # 경로확인/생성    
    zip = zipfile.ZipFile(toPath, 'w')
    zip.write(fromPath, compress_type=zipfile.ZIP_DEFLATED)    

## Convert Process

In [9]:

'''
대용량파일의 경우 분할하여 저장..(카탈로그 내 피드가 여러개인 경우)
'''
async def execute():
    # ep download
    # await epDownload(config['ep']['url'], config['ep']['path'])

    # total
    totalCount = 0
    totalChunkCount = 0
    for num, chunkDF in enumerate(epLoad()):
        totalCount = totalCount + chunkDF.shape[0]
        totalChunkCount = num # 최종 num 저장됨
        
        del[[chunkDF]]
        gc.collect()

    # segmentation
    feedCount = len(config['info']['feed_id']) # 카탈로그당 나눈 피드갯수
    chunkCountPerFeed = int(round(totalChunkCount/feedCount)) # 피드하나당 chunk갯수
    print(f'totalChunkCount : {totalChunkCount}')
    print(f'feedCount : {feedCount}')
    print(f'chunkCountPerFeed : {chunkCountPerFeed}')


    # chunk read
    count = 0
    feedIndex=0
    for num, chunkDF in enumerate(epLoad()):
        # print(chunkDF.head())
        count = count + chunkDF.shape[0]
        print(f'{round(num/totalChunkCount*100)}%', end='...')

        # filter
        chunkDF = convertFilter(chunkDF)

        # write        
        segmentIndex = num % chunkCountPerFeed # feed_id당 배분될 chunk만큼의 index 생성
        # print(f'segmentIndex : {segmentIndex}')
        
        if segmentIndex == 0: # 0일때마다 feed 분할
            feed_id = config['info']['feed_id'][feedIndex]
            feedSecPath = str(config['feed']['path']).replace('.tsv', f'_{feed_id}.tsv')
            if feedIndex < len(config['info']['feed_id'])-1:
                feedIndex = feedIndex + 1
    
            
        await feedWrite(segmentIndex, feedSecPath, chunkDF) #feed분할 적용       
        # await feedWrite(num, config['feed']['path'], chunkDF)
            
        del[[chunkDF]]
        gc.collect()

        # break

    # 속도가 아쉬움..압축을 안한다면?
    # catalog내 feed분할 업로드 위한 압축
    catalog_id = config['info']['catalog_id']
    for num, feed_id in enumerate(config['info']['feed_id']):
        feedPath = f'C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/feed_{catalog_id}_{feed_id}.tsv'
        zipPath = f'C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/feed_{catalog_id}_{feed_id}.zip'
        zipped(feedPath, zipPath)

        # facebook api upload
        # print(await uploadAPI(feed_id, feedZipPath))

In [12]:
await execute()

totalChunkCount : 111
feedCount : 5
chunkCountPerFeed : 22
0%...1%...2%...3%...4%...5%...5%...6%...7%...8%...9%...10%...11%...12%...13%...14%...14%...15%...16%...17%...18%...19%...20%...21%...22%...23%...23%...24%...25%...26%...27%...28%...29%...30%...31%...32%...32%...33%...34%...35%...36%...37%...38%...39%...40%...41%...41%...42%...43%...44%...45%...46%...47%...48%...49%...50%...50%...51%...52%...53%...54%...55%...56%...57%...58%...59%...59%...60%...61%...62%...63%...64%...65%...66%...67%...68%...68%...69%...70%...71%...72%...73%...74%...75%...76%...77%...77%...78%...79%...80%...81%...82%...83%...84%...85%...86%...86%...87%...88%...89%...90%...91%...92%...93%...94%...95%...95%...96%...97%...98%...99%...100%...

In [62]:
await epDownload(config['ep']['url'], config['ep']['path'])

CancelledError: 

In [37]:
0%0

ZeroDivisionError: integer division or modulo by zero

In [38]:
0/1

0.0

In [41]:
# 중복제거 test
# 중복제거 test
tempPath = 'C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/225456985373646/feed_225456985373646_temp.tsv'
tempLoad = pd.read_csv(tempPath,
            header=0, # header row                            
            dtype=str, # string type 인식
            sep='\t', # 명시
            # lineterminator='\r',
            error_bad_lines=False, # error skip
            encoding='utf-8')


In [42]:
print(tempLoad.shape[0])

7317167

In [44]:
tempLoad_1 = tempLoad.drop_duplicates(['id'], keep='first')
print(tempLoad_1.shape[0])

6398343


In [90]:
testDict = {"id1":{"path":"value1"}, "id2":{"path":"value2"}}
print(len(testDict.keys()))    
for value in testDict.keys() :
    print(testDict[value]['path'])
    

2
value1
value2


In [212]:
# 한 리스트에 몇개씩 담는가?
import math
def getSegmentIndexMap(feedlen=1): # 2,4,10 으로만 분할..
    for i in range(10):
        
        print(10/(i+1), 10%(i+1))
    # return result

getSegmentIndexMap(2)

10.0 0
5.0 0
3.3333333333333335 1
2.5 2
2.0 0
1.6666666666666667 4
1.4285714285714286 3
1.25 2
1.1111111111111112 1
1.0 0


In [199]:
l = [0,1,2,3,4,5,6,7,8,9]

print(l[0:10]) # 1
print(l[0:5],l[5:10]) # 2
print(l[0:3],l[3:6],l[6:10]) #3
print(l[0:3],l[3:6],l[6:9],l[9:10]) # 4
print(l[0:2],l[2:4],l[4:6],l[6:8],l[8:10]) # 5
print(l[0:2],l[2:4],l[4:6],l[6:8],l[8:9],l[9:10]) # 6
print(l[0:2],l[2:4],l[4:6],l[6:7],l[7:8],l[8:9],l[9:10]) # 7
print(l[0:2],l[2:4],l[4:5],l[5:6],l[6:7],l[7:8],l[8:9],l[9:10]) # 8
print(l[0:2],l[2:3],l[3:4],l[4:5],l[5:6],l[6:7],l[7:8],l[8:9],l[9:10]) # 9
print(l[0:1],l[1:2],l[2:3],l[3:4],l[4:5],l[5:6],l[6:7],l[7:8],l[8:9],l[9:10]) # 10


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4] [5, 6, 7, 8, 9]
[0, 1, 2] [3, 4, 5] [6, 7, 8, 9]
[0, 1, 2] [3, 4, 5] [6, 7, 8] [9]
[0, 1] [2, 3] [4, 5] [6, 7] [8, 9]
[0, 1] [2, 3] [4, 5] [6, 7] [8] [9]
[0, 1] [2, 3] [4, 5] [6] [7] [8] [9]
[0, 1] [2, 3] [4] [5] [6] [7] [8] [9]
[0, 1] [2] [3] [4] [5] [6] [7] [8] [9]
[0] [1] [2] [3] [4] [5] [6] [7] [8] [9]


In [214]:
import pandas as pd
df = pd.read_csv('C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/ep/ep_432089861336082.tsv',
        nrows=None,        
        header=0, # header row                            
        dtype=str, # string type 인식
        sep='\t', # 명시
        # lineterminator='\r',
        error_bad_lines=False, # error skip    
        encoding='utf-8')

In [217]:
df[df['id'].str[-1:].isin(['9','6'])]

,id,title,price_pc,link,image_link,category_name1,category_name2,category_name3,category_name4,brand,maker,origin,coupon,point,review_count,shipping
0,021379,[강성은명과] 동백 세트 639g,39000,https://www.hellonature.co.kr/fdp001.do?goTo=d...,https://img-ex.hellonature.co.kr/images/ex/PGD...,커피·음료·간식,과자류,기획전,음료/차/다과,강성은명과,생산자 : 강성은명과 / 서울시 서대문구,상세페이지참조,NaN,NaN,14,2500
2,011279,[사랑과정성] 돈가스소스 230g,3600,https://www.hellonature.co.kr/fdp001.do?goTo=d...,https://img-ex.hellonature.co.kr/images/ex/PGD...,양념·오일·면,소스·드레싱·육수,NaN,NaN,사랑과정성,제조원: (주)사랑과정성,상세페이지참조,NaN,NaN,857,2500
9,023339,고수 30g,1490,https://www.hellonature.co.kr/fdp001.do?goTo=d...,https://img-ex.hellonature.co.kr/images/ex/PGD...,채소,쌈·샐러드,기획전,추천재료,신선한 채소,취급자: 사회적기업 청보 농업회사법인(주),국내산,NaN,NaN,329,2500
15,028616,[맥케인] 이모티콘 감자튀김 1.25kg,9900,https://www.hellonature.co.kr/fdp001.do?goTo=d...,https://img-ex.hellonature.co.kr/images/ex/PGD...,국·반찬·요리,만두·튀김·냉동식품,키즈,FUN,맥케인,생산자 : MCCAIN / 수입자 : (주)조흥,벨기에,NaN,NaN,66,2500
21,101556,[랩신] 토탈 핸드워시 250mL,5900,https://www.hellonature.co.kr/fdp001.do?goTo=d...,https://img-ex.hellonature.co.kr/images/ex/PGD...,생활·선물세트,생활·위생용품,NaN,NaN,랩신,(주)태동씨엔에스 / 경기도 안산시,상세페이지참조,NaN,NaN,2,2500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2859,022446,1등급 한우갈비세트,220000,https://www.hellonature.co.kr/fdp001.do?goTo=d...,https://img-ex.hellonature.co.kr/images/ex/PGD...,생활·선물세트,선물세트,기획전,정육,사랑해우,상세페이지참조,국내산,NaN,NaN,0,0
2865,101639,[장수이야기] 십이전대보차 70ml*10개입,25000,https://www.hellonature.co.kr/fdp001.do?goTo=d...,https://img-ex.hellonature.co.kr/images/ex/PGD...,커피·음료·간식,차,기획전,헬로네이처 베스트템,장수이야기,"제조원 : 농업회사법인 장수이야기 주식회사, 경기도 김포",상세페이지참조,NaN,NaN,15,2500
2871,028459,[미나티] 초코퐁당 뚱카롱 3종 세트 110g,8700,https://www.hellonature.co.kr/fdp001.do?goTo=d...,https://img-ex.hellonature.co.kr/images/ex/PGD...,베이커리·유제품,케이크·파이·디저트류,기획전,디저트,미나티,미나티 / 경기도 의왕시 내손중앙6길 77(1층 내손동),상세페이지참조,NaN,NaN,30,2500
2875,028436,[테일러팜스] 유기농 푸룬 주스 946ml,11000,https://www.hellonature.co.kr/fdp001.do?goTo=d...,https://img-ex.hellonature.co.kr/images/ex/PGD...,커피·음료·간식,주스,기획전,건강음료&주스,테일러팜스,"제조원: (주)테일러팜스 / 부산시 강서구, 판매원: (주)테일러팜스 / 경기도 고양시",상세페이지참조,NaN,NaN,48,2500


In [51]:
my_list = ['0','1','2','3','4','5','6','7','8','9']
def divide_list(l, n): 
    # 리스트 l의 길이가 n이면 계속 반복
    for i in range(0, len(l), n): 
        yield l[i:i + n] 
# 한 리스트에 몇개씩 담을지 결정
n = 5
result = list(divide_list(my_list, n))
'print(result)

[['0', '1', '2'], ['3', '4', '5'], ['6', '7', '8'], ['9']]


In [227]:
for i in range(10) :
    locals()[f'temp_{i}'] = i


1
1
1
1
1
1
1
1
1
1


In [233]:
temp1 = pd.DataFrame()
temp2 = pd.DataFrame([{'a':3,'b':3,'c':3}, {'a':4,'b':4,'c':4}])
temp3 = pd.concat([temp1,temp2], ignore_index=True)
temp3

,a,b,c
0,3,3,3
1,4,4,4


In [232]:
temp1

,a,b,c
0,1,1,1
1,2,2,2
